# INSTITUTO DE EDUCAÇÃO SUPERIOR DE BRASÍLIA – IESB
# PÓS-GRADUAÇÃO EM CIÊNCIA DE DADOS
# Softwares para uso em Ciência de Dados III
# Prof.: Thiago Baroni
# Aluno: André dos Santos Feitosa
# Matrícula:1931133005

# Contexto

Da Organização Mundial da Saúde - Em 31 de dezembro de 2019, a OMS foi alertada sobre vários casos de pneumonia na cidade de Wuhan, província de Hubei, na China. O vírus não encontrou nenhum outro vírus conhecido. Isso causou preocupação porque, quando um vírus é novo, não sabemos como ele afeta as pessoas.
Portanto, as informações diárias das pessoas afetadas podem fornecer informações interessantes quando disponibilizadas para a comunidade mais ampla de ciência de dados.
A Johns Hopkins University criou um excelente painel usando os dados de casos afetados. Os dados são extraídos das planilhas do Google associadas e disponibilizados aqui.




# Conteúdo

O novo coronavírus de 2019 (2019-nCoV) é um vírus (mais especificamente, um coronavírus) identificado como a causa de um surto de doença respiratória detectado pela primeira vez em Wuhan, China. No início, muitos dos pacientes do surto em Wuhan, na China, teriam algum vínculo com um grande mercado de frutos do mar e animais, sugerindo a disseminação de animais para pessoas. No entanto, um número crescente de pacientes supostamente não teve exposição ao mercado de animais, indicando a ocorrência de disseminação de pessoa para pessoa. No momento, não está claro o quão fácil ou sustentável esse vírus está se espalhando entre as pessoas - CDC
Este conjunto de dados possui informações diárias sobre o número de casos afetados, mortes e recuperação do novo coronavírus de 2019. Observe que esses são dados de séries temporais e, portanto, o número de casos em um determinado dia é o número acumulado.
Os dados estão disponíveis a partir de 22 de janeiro de 2020.


In [ ]:
# Dados das Colunas

#SNo - Númeração da linha
#ObservationDate - Data da Observação 
#Province/State - Provincia/Estado
#Country/Region - Pais/Região
#Last Update - ùltima Atuliazação
#Confirmed - Casos Confirmados
#Deaths - Mortes
#Recovered - Casos Recuperados 


In [ ]:
# Colunas Renomeadas

#Nº - Númeração da linha
#Data da Observacao - Data da Observação 
#Provincia_estado - Provincia/Estado
#Pais_regiao - Pais/Região
#Ultima_atlz - ùltima Atuliazação
#Confirmada - Casos Confirmados
#Mortes - Mortes
#Recuperada - Casos Recuperados 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        import warnings
warnings.filterwarnings("ignore")        
        
        

# Any results you write to the current directory are saved as output.

In [ ]:
# importação das bibliotecas

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.ar_model import AR
from random import random
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [ ]:
# Importação o arquivo e definição do nome

df = pd.read_csv("/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv")

df.head(100)


In [ ]:
# Renomeando as colunas

df.columns = ['Nº', 'DATA_OBS', 'PROVINCIA_ESTADO', 'PAIS_REGIAO', 'ULTIMA_ATLZ', 'CONFIRMADO', 
              'MORTES', 'RECUPERADOS']        


df.head(500)

In [ ]:
# Seleção dos dados do Brasil

df_brasil = df[df['PAIS_REGIAO'] == 'Brazil']

df_brasil.head()


In [ ]:
# Criando a coluna calculada com dados dos contaminados


df_brasil['CONTAMINADOS'] = df_brasil['CONFIRMADO'] - (df_brasil['MORTES'] - df_brasil['RECUPERADOS'])

df_brasil.head(50)


In [ ]:
# Dados descritivos da nova coluna - Contaminados

df_brasil.CONTAMINADOS .describe()

In [ ]:
# Gráfico demostrativo BOX SPLOT da Variavel contaminação

df_brasil['CONTAMINADOS'].plot.box()

In [ ]:
# Gráfico demostrativo da evolução pela média dos casos contaminados

df_brasil.groupby('DATA_OBS')['CONTAMINADOS'].mean().plot()

In [ ]:
# Gráfico evolutivo dos casos contaminados

sns.pairplot(x_vars='DATA_OBS',y_vars='CONTAMINADOS', data=df_brasil, height=10)

In [ ]:
# Separando as colunas que irão fazer parte dos modelos


df_contaminados = df_brasil.drop(columns=['Nº','PROVINCIA_ESTADO','PAIS_REGIAO','ULTIMA_ATLZ','CONFIRMADO','MORTES','RECUPERADOS'])
df_contaminados.head()



In [ ]:
# Verificando os formatos das colunas
#OBS: Será necessário alterar a coluna DATA_OBS para o formato datetime

df_contaminados.info()

In [ ]:
# Convertendo a coluna DATA_OBS em datetime

df_contaminados['DATA_OBS'] = pd.to_datetime(df['DATA_OBS'])

df_contaminados.info()

In [ ]:
df_contaminados.head()

In [ ]:
#Atribuindo a coluna CONTAMINADOS no index da base

df_contaminados.index = df_contaminados['DATA_OBS']
df_contaminados.drop('DATA_OBS',inplace=True, axis=1)
df_contaminados.head()

In [ ]:
df_contaminados.head()

# Modelos e Resultados

In [ ]:
# Plotando o gráfico de evolução para inserir a linha de regressão.

plt.plot(df_contaminados)
plt.title('Evolução')
plt.ylabel('DATA_OBS')
plt.xlabel('CONTAMINADOS')
plt.show()

In [ ]:
# Inclusão da linha de regressão


X = [i for i in range(0, len(df_contaminados))]
X = np.reshape(X, (len(X), 1))
y = df_contaminados
modelo = LinearRegression()
modelo.fit(X, y)
modelo.coef_
tendencia = modelo.predict(X)

plt.plot(df_contaminados.values, color='blue',label='Original')
plt.title('Evolução dos contaminados pelo COVID - 19')
plt.ylabel('DATA_OBS')
plt.xlabel('CONTAMINADOS')
plt.plot(tendencia,color='red', label='Tendência' )
plt.show()

# Modelo Arima


In [ ]:
# Estatísticas do modelo ARIMA

modelo = ARIMA(df_contaminados, order=(5,1, 0))
modelo_treinado = modelo.fit()
print(modelo_treinado.summary())

In [ ]:
#Previsão

fig = modelo_treinado.plot_predict(start=1,end=60)
plt.title('Previsão de Contaminados, 60 dias utilizando o modelo ARIMA')
plt.ylabel('CONTAMINADOS')
plt.xlabel('DATA_OBS')




# Modelo Arma

In [ ]:
#Arma - modelos auto-regressivos de médias móveis

arma_mod = sm.tsa.ARMA(df_contaminados, (3,0)).fit(disp=False)
print(arma_mod.summary())


In [ ]:
#Previssão para 60 dias

fig = arma_mod.plot_predict(start=1, end=60)
plt.title('Previssão do COVID para 60 dias, utilizando o modelo ARMA')
plt.ylabel('CONTAMINADOS')
plt.xlabel('DATA_OBS')


# MODELO NAIVE

In [ ]:
#Modelo SNaive, com sazonalidade

snaive = ARIMA(df_contaminados, order=(0, 1, 0))
snaive_treinado = snaive.fit()
print(snaive_treinado.summary())

In [ ]:
snaive_treinado.plot_predict(start=1,end=60)
plt.title('Previssão do COVID para 60 dias, utilizando o modelo NAIVE')
plt.show()

In [ ]:
#Modelo Naive, sem sazonalidade

naive = ARIMA(df_contaminados, order=(0, 0, 0))
naive_treinado = naive.fit()
print(naive_treinado.summary())

In [ ]:
plt.rcParams.update({'figure.figsize':(9,3)})
naive_treinado.plot_predict(start=1,end=90)
plt.title('Previssão do COVID para 90 dias, utilizando o modelo NAIVE')
plt.show()